In [1]:
import torch
import torchaudio

In [2]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(torchaudio.__version__)
print(device)

1.12.0
0.12.0
cuda


In [3]:
bundle = torchaudio.pipelines.WAV2VEC2_BASE

print("Sample Rate:", bundle.sample_rate)

model = bundle.get_model().to(device)

print(model.__class__)

Sample Rate: 16000
<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>


In [4]:
import os

input_training_file = 'train_files/training.txt'
input_wavs_dir = 'train_files/wavs'
with open(input_training_file, 'r', encoding='utf-8') as fi:
        training_files = [os.path.join(input_wavs_dir, x.split('|')[0] + '.wav')
                          for x in fi.read().split('\n') if len(x) > 0]

training_files

['train_files/wavs/LJ001-0001.wav',
 'train_files/wavs/LJ001-0002.wav',
 'train_files/wavs/LJ001-0003.wav',
 'train_files/wavs/LJ001-0004.wav',
 'train_files/wavs/LJ001-0005.wav',
 'train_files/wavs/LJ001-0006.wav',
 'train_files/wavs/LJ001-0007.wav',
 'train_files/wavs/LJ001-0008.wav',
 'train_files/wavs/LJ001-0009.wav',
 'train_files/wavs/LJ001-0010.wav']

In [12]:
output_dir = 'wav2vec'

for file in training_files:
    waveform, sample_rate = torchaudio.load(file)
    waveform = waveform.to(device)

    if sample_rate != bundle.sample_rate:
        waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

    # with torch.inference_mode():
    #     features, _ = model.extract_features(waveform)

    res, _ = model(waveform)

    output_file = os.path.join(output_dir, file.split('/')[-1].split('.')[0] + '.pt')

    torch.save(res, output_file)
    print(res.size())

torch.Size([1, 482, 768])
torch.Size([1, 94, 768])
torch.Size([1, 483, 768])
torch.Size([1, 256, 768])
torch.Size([1, 405, 768])
torch.Size([1, 283, 768])
torch.Size([1, 419, 768])
torch.Size([1, 88, 768])
torch.Size([1, 377, 768])
torch.Size([1, 440, 768])


In [19]:
output_dir = 'train_files/wavs/LJ001-0001.wav'
output_dir1 = 'wav2vec'
print(os.path.join(output_dir1, os.path.splitext(os.path.split(output_dir)[-1])[0] + '.pt'))

wav2vec/LJ001-0001.pt


In [40]:
import numpy as np

file = 'wav2vec/LJ001-0001.pt'

r = torch.load(file)

print(r.size())

torch.Size([1, 482, 768])


In [41]:
r = r.squeeze()

In [46]:
r = torch.transpose(r, 0, 1)
r.size()

torch.Size([768, 482])

In [48]:
r.unsqueeze(0).size()

torch.Size([1, 768, 482])